In [2]:
from dotenv import load_dotenv
import os
import pandas as pd
import sqlite3
import requests
import json
from pandas import json_normalize
import pandas as pd

# Load env. variables from .env file
load_dotenv()
api_key = os.getenv("RAPIDAPI_KEY")
api_host = os.getenv("RAPIDAPI_HOST")
db_path = os.getenv("db_path")
json_path = os.getenv("json_path")

In [3]:
# Load JSON file from data/json_files/season
with open("C:/Users/Lavoro/Desktop/AIBetting/AIbetting/core/data/json_files/2021/matches_info.json") as json_file:
    data = json.load(json_file)
    print(data)

{'get': 'fixtures', 'parameters': {'date': '2021-01-01'}, 'errors': [], 'results': 99, 'paging': {'current': 1, 'total': 1}, 'response': [{'fixture': {'id': 654209, 'referee': None, 'timezone': 'UTC', 'date': '2021-01-01T05:40:00+00:00', 'timestamp': 1609479600, 'periods': {'first': 1609479600, 'second': 1609483200}, 'venue': {'id': None, 'name': 'Japan National Stadium', 'city': 'Tokyo'}, 'status': {'long': 'Match Finished', 'short': 'FT', 'elapsed': 90}}, 'league': {'id': 102, 'name': 'Emperor Cup', 'country': 'Japan', 'logo': 'https://media.api-sports.io/football/leagues/102.png', 'flag': 'https://media.api-sports.io/flags/jp.svg', 'season': 2020, 'round': 'Final'}, 'teams': {'home': {'id': 294, 'name': 'Kawasaki Frontale', 'logo': 'https://media.api-sports.io/football/teams/294.png', 'winner': True}, 'away': {'id': 293, 'name': 'Gamba Osaka', 'logo': 'https://media.api-sports.io/football/teams/293.png', 'winner': False}}, 'goals': {'home': 1, 'away': 0}, 'score': {'halftime': {'hom

In [4]:
df1 = pd.json_normalize(data, 'response', sep='_') # Extract the response data
df1.head()

,fixture_id,fixture_referee,fixture_timezone,fixture_date,fixture_timestamp,fixture_periods_first,fixture_periods_second,fixture_venue_id,fixture_venue_name,fixture_venue_city,...,goals_home,goals_away,score_halftime_home,score_halftime_away,score_fulltime_home,score_fulltime_away,score_extratime_home,score_extratime_away,score_penalty_home,score_penalty_away
0,654209,None,UTC,2021-01-01T05:40:00+00:00,1609479600,1.609480e+09,1.609483e+09,NaN,Japan National Stadium,Tokyo,...,1.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN
1,651308,None,UTC,2021-01-01T07:00:00+00:00,1609484400,1.609484e+09,1.609488e+09,608.0,Addis Ababa Stadium,Addis Abeba,...,3.0,2.0,NaN,NaN,3.0,2.0,NaN,NaN,NaN,NaN
2,653168,None,UTC,2021-01-01T09:00:00+00:00,1609491600,1.609492e+09,1.609495e+09,NaN,None,None,...,1.0,2.0,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN
3,738437,None,UTC,2021-01-01T09:00:00+00:00,1609491600,1.609492e+09,1.609495e+09,NaN,None,None,...,2.0,3.0,NaN,NaN,2.0,3.0,NaN,NaN,NaN,NaN
4,687508,None,UTC,2021-01-01T10:00:00+00:00,1609495200,1.609495e+09,1.609499e+09,3798.0,Bangabandhu National Stadium,Dhaka,...,0.0,2.0,NaN,NaN,0.0,2.0,NaN,NaN,NaN,NaN


In [23]:
df_players_season_stats = pd.json_normalize(df1["statistics"])
df_players_season_stats.head()

,0,1
0,"{'team.id': 577, 'team.name': 'FK Kukesi', 'te...",None
1,"{'team.id': 577, 'team.name': 'FK Kukesi', 'te...",None
2,"{'team.id': 577, 'team.name': 'FK Kukesi', 'te...",None
3,"{'team.id': 577, 'team.name': 'FK Kukesi', 'te...","{'team.id': 577, 'team.name': 'FK Kukesi', 'te..."
4,"{'team.id': 577, 'team.name': 'FK Kukesi', 'te...",None


In [31]:
# Converti la stringa JSON in un dizionario Python
data = json.loads(data)

# Estrai i dati di risposta
response_data = data['response'][0]

# Normalizza il JSON annidato in un DataFrame
df = json_normalize(response_data)

# Visualizza il DataFrame
df.head()

TypeError: the JSON object must be str, bytes or bytearray, not dict